In [5]:
import numpy as np
from torchvision import datasets
from sklearn.decomposition import PCA
import pickle

import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

In [8]:
def create_2d_mnist():
    flat_shape = 784
    pca_shape = 2
    
    mnist = datasets.MNIST('../data', train=True, download=True)
    
    X = mnist.train_data.reshape(-1, flat_shape).numpy()
    y = mnist.train_labels.numpy()

    pca = PCA(n_components=pca_shape)
    X_pca = pca.fit_transform(X)
    
    return X_pca, y

In [74]:
X_full, y_true_full = create_2d_mnist()

/home/rv/.local/lib/python3.10/site-packages/torchvision/datasets/mnist.py:75: UserWarning:

train_data has been renamed data

/home/rv/.local/lib/python3.10/site-packages/torchvision/datasets/mnist.py:65: UserWarning:

train_labels has been renamed targets



In [75]:
random_mask = np.full(full_size, False)
random_mask[:10000] = True
np.random.shuffle(random_mask)
X_full = X_full[random_mask]
y_true_full = y_true_full[random_mask]

In [76]:
full_size = y_true_full.shape[0]
unlearn_size = int(y_true_full.shape[0] * 0.2)
retain_size = full_size - unlearn_size
print(retain_size, unlearn_size)

8000 2000


In [77]:
unlearn_mask = np.full(full_size, False)
unlearn_mask[:unlearn_size] = True
np.random.shuffle(unlearn_mask)
retain_mask = ~unlearn_mask

In [78]:
X_retain, y_true_retain = X_full[retain_mask], y_true_full[retain_mask]
X_unlearn, y_true_unlearn = X_full[unlearn_mask], y_true_full[unlearn_mask]

In [79]:
class KMeans_InitMethods:
    @staticmethod
    def forgy(X, row_count, n_clusters):
        np.random.seed(0)
        return X [ np.random.choice(row_count, size=n_clusters, replace=False) ]

    @staticmethod
    def macqueen(X, n_clusters):
        return X [:n_clusters]

    @staticmethod
    def maximin(X, n_clusters):
        X_ = np.copy(X)
        initial_centers = np.zeros((n_clusters, X_.shape[1]))
        X_norms = np.linalg.norm(X_, axis = 1)
        X_norms_max_i = X_norms.argmax()
        initial_centers[0] = X_[X_norms_max_i]
        X_ = np.delete(X_, X_norms_max_i, axis = 0)
        for i in range(1, n_clusters):
            distances = np.zeros((X_.shape[0], i))
            for index, center in enumerate(initial_centers[:i]):
                distances[:, index] = np.linalg.norm(X_ - center, axis = 1)

            max_min_index = distances.min(axis = 1).argmax()

            initial_centers[i] = X_[max_min_index]
            X_ = np.delete(X_, max_min_index, axis = 0)

        return initial_centers

    @staticmethod
    def var_part(X, n_clusters):
        X_ = np.append(X, np.zeros(X.shape[0])[:, np.newaxis], axis = 1)
        initial_centers = np.zeros((n_clusters, X.shape[1]))

        cluster_i = 1
        while cluster_i != n_clusters:
            within_clusters_sum_squares = np.zeros(cluster_i)
            for j in range(cluster_i):
                cluster_members = X_[ X_[:, -1] == j ]
                cluster_mean = cluster_members.mean(axis = 0)
                within_clusters_sum_squares[j] = np.linalg.norm(cluster_members - cluster_mean, axis = 1).sum()

            # Cluster which has greatest SSE
            max_sse_i = within_clusters_sum_squares.argmax()
            X_max_sse_i = X_[:, -1] == max_sse_i
            X_max_sse = X_ [ X_max_sse_i ]

            variances, means = X_max_sse.var(axis = 0), X_max_sse.mean(axis = 0)
            max_variance_i = variances.argmax()
            max_variance_mean = means [ max_variance_i ]

            X_smaller_mean = X_max_sse[:, max_variance_i] <= max_variance_mean
            X_greater_mean = X_max_sse[:, max_variance_i] > max_variance_mean

            initial_centers[max_sse_i] = X_max_sse [ X_smaller_mean ].mean(axis = 0)[:-1]
            initial_centers[cluster_i] = X_max_sse [ X_greater_mean ].mean(axis = 0)[:-1]

            X_[ (X_max_sse_i) & (X_ [:, max_variance_i] <= max_variance_mean), -1] = cluster_i
            X_[ (X_max_sse_i) & (X_ [:, max_variance_i] > max_variance_mean), -1] = max_sse_i

            cluster_i += 1  

        return initial_centers

class KMeans:

    def __init__(self, n_clusters = 3, tolerance = 0.01, max_iter = 100, runs = 1, init_method="forgy"):
        self.n_clusters = n_clusters
        self.tolerance = tolerance
        self.cluster_means = np.zeros(n_clusters)
        self.max_iter = max_iter
        self.init_method = init_method

        # There is no need to run the algorithm multiple times if the 
        # initialization method is not a random process
        self.runs = runs if init_method == 'forgy' else 1
        
    def fit(self, X_values: np.ndarray):
        row_count, col_count = X_values.shape
        
        X_labels = np.zeros(row_count)
        
        costs = np.zeros(self.runs)
        all_clusterings = []
        
        for i in range(self.runs):
            cluster_means =  self.__initialize_means(X_values, row_count)
            cluster_means_history = [cluster_means]
            
            for _ in range(self.max_iter):            
                previous_means = np.copy(cluster_means)
                
                distances = self.__compute_distances(X_values, cluster_means, row_count)
            
                X_labels = self.__label_examples(distances)
            
                cluster_means = self.__compute_means(X_values, X_labels, col_count)
                cluster_means_history.append(cluster_means)

                clusters_not_changed = np.abs(cluster_means - previous_means) < self.tolerance
                if np.all(clusters_not_changed) != False:
                    break
            
            X_values_with_labels = np.append(X_values, X_labels[:, np.newaxis], axis = 1)
            
            cluster_means_history = np.array(cluster_means_history)
            all_clusterings.append( (cluster_means, X_values_with_labels, cluster_means_history) )
            costs[i] = self.__compute_cost(X_values, X_labels, cluster_means)
        
        best_clustering_index = costs.argmin()

        self.cost_ = costs[best_clustering_index]
        
        return all_clusterings[best_clustering_index]
        
    def __initialize_means(self, X, row_count):
        if self.init_method == 'forgy':
            return KMeans_InitMethods.forgy(X, row_count, self.n_clusters)
        elif self.init_method == 'maximin':
            return KMeans_InitMethods.maximin(X, self.n_clusters)
        elif self.init_method == 'macqueen':
            return KMeans_InitMethods.macqueen(X, self.n_clusters)
        elif self.init_method == 'var_part':
            return KMeans_InitMethods.var_part(X, self.n_clusters)
        else:
            raise Exception('The initialization method {} does not exist or not implemented'.format(self.init_method))
        
    def __compute_distances(self, X, cluster_means, row_count):
        distances = np.zeros((row_count, self.n_clusters))
        for cluster_mean_index, cluster_mean in enumerate(cluster_means):
            distances[:, cluster_mean_index] = np.linalg.norm(X - cluster_mean, axis = 1)
            
        return distances
    
    def __label_examples(self, distances):
        return distances.argmin(axis = 1)
    
    def __compute_means(self, X, labels, col_count):
        cluster_means = np.zeros((self.n_clusters, col_count))
        for cluster_mean_index, _ in enumerate(cluster_means):
            cluster_elements = X [ labels == cluster_mean_index ]
            if len(cluster_elements):
                cluster_means[cluster_mean_index, :] = cluster_elements.mean(axis = 0)
                
        return cluster_means
    
    def __compute_cost(self, X, labels, cluster_means):
        cost = 0
        for cluster_mean_index, cluster_mean in enumerate(cluster_means):
            cluster_elements = X [ labels == cluster_mean_index ]
            cost += np.linalg.norm(cluster_elements - cluster_mean, axis = 1).sum()
        
        return cost

In [80]:
kmeans = KMeans(n_clusters = 10, max_iter=1000)

In [81]:
(_, clusters_full, history_full) = kmeans.fit(X_full)

In [82]:
import plotly.graph_objects as go
fig = go.Figure()

for cluster_id in range(0, 10):
    fig.add_trace(
        go.Scatter(x=history_full[:,cluster_id, 0], 
                   y=history_full[:,cluster_id, 1], 
                   mode='lines+markers', name=cluster_id, marker=dict(size=3))
    )
fig.show()

In [87]:
(cluster_centers_retain, clusters_retain, history_retain) = kmeans.fit(X_retain)

In [84]:
import plotly.graph_objects as go
fig = go.Figure()

for cluster_id in range(0, 10):
    fig.add_trace(
        go.Scatter(x=history_retain[:,cluster_id, 0], 
                   y=history_retain[:,cluster_id, 1], 
                   mode='lines+markers', name=cluster_id, marker=dict(size=3))
    )
fig.show()

In [85]:
px.scatter(x=X_full[:, 0], y=X_full[:, 1], color=clusters_full[:, 2])

In [96]:
fig1 = px.scatter(x=X_retain[:, 0], y=X_retain[:, 1], color=clusters_retain[:, 2])
fig2 = px.scatter(x=cluster_centers_retain[0], y=cluster_centers_retain[1])
fig = go.Figure(data=fig1.data + fig2.data)
fig.show()

In [182]:
clusters.shape

(10, 2)

In [209]:
cluster_means_history.shape

(101, 10, 2)

In [210]:
cluster_means_history[:, 0, 0]

array([ 706.43641356,  839.55849146,  991.52876556, 1050.37903504,
       1091.46939731, 1124.05586774, 1148.33646757, 1169.63066252,
       1185.39325239, 1198.94394103, 1209.35027945, 1218.2040961 ,
       1226.77850199, 1235.56395748, 1243.02993265, 1250.49796053,
       1256.96806805, 1263.48812131, 1270.17130414, 1277.54772766,
       1283.55993223, 1288.6388858 , 1293.29837633, 1296.76800732,
       1300.95050309, 1305.95705284, 1310.51151078, 1314.21788666,
       1317.04394148, 1320.02855221, 1323.01314447, 1325.20277064,
       1327.03575287, 1328.96091479, 1330.69713366, 1332.28587652,
       1333.81272637, 1335.55318861, 1336.38356774, 1337.50194577,
       1338.07597885, 1338.78524075, 1339.49045527, 1340.12686263,
       1340.44101055, 1340.87851776, 1341.17690787, 1341.94912524,
       1342.28696224, 1342.61325108, 1343.16424599, 1343.49018557,
       1343.58959092, 1343.70031522, 1343.58951593, 1343.75516042,
       1343.84625989, 1344.07312125, 1344.22492087, 1344.46619

In [206]:
cluster_means_history = np.array(cluster_means_history)

In [223]:
px.line(x=cluster_means_history[:, 0, 0], y=cluster_means_history[:, 0, 1])

In [225]:
cluster_means_history.shape

(101, 10, 2)

In [ ]:
cluster_df = pd.Da

In [218]:
import plotly.graph_objects as go

fig1 = px.scatter(x=X_full[:, 0], y=X_full[:, 1], color=data_with_clusters[:, 2])
fig2 = px.line(x=cluster_means_history[:, 0, 0], y=cluster_means_history[:, 0, 1])

fig = go.Figure(data = fig1.data + fig2.data)
fig.show()

ValueError: 
    Invalid value of type 'builtins.int' received for the 'shape' property of scatter.line
        Received value: 2

    The 'shape' property is an enumeration that may be specified as:
      - One of the following enumeration values:
            ['linear', 'spline', 'hv', 'vh', 'hvh', 'vhv']